## Esame 17 ottobre

L'esame sarà valutato

- verificando la correttezza dell'output generato dal notebook
- analizzando il codice sottomesso

⚠ Prima di iniziare

1. andare in basso a sinistra in Settings - Admin Console
2. cliccare su Add User e aggiungere `m.santoni@itsrizzoli.it`

L'esame è composto da 3 esercizi descritti in calce. Al completamento dei 3 esercizi,

- esporta il tuo notebook tramite File - Export - IPython Notebook
- caricalo su [questo indirizzo](https://docs.google.com/forms/d/e/1FAIpQLSelUXQ9gJb83SUYVCjbvXdFmo6fQMhTgsbnMJKEjyqOKSYYfg/viewform?usp=sf_link) entro la fine del tempo prestabilito ⏳


### Vinho Verde 🍾

Nella cartella `/databricks-datasets/wine-quality` si trova il file `winequality-white.csv`. Il dataset descrive varianti del vino bianco portoghese _Vinho Verde_. Notare che il separatore delle colonne del file è `;` e non `,`.

#### Parte 1
  - contare il numero di varianti di vino per ogni livello di `quality` (valore tra `1` e `8`). Ordinare quindi il conteggio per `quality` crescente. Esempio

| quality | count |
|--------|------|
| 1 | 123 |
| 2 | 456 |
| ... | ... |

  - salvare il risultato in formato parquet nel seguente path `/exam/221017_wine_part_1.parquet` (⚠ i nomi delle colonne devono coincidere con quelli indicati nella traccia)

#### Parte 2
- considerare solo le varianti di vino il cui zucchero residuo (`residual sugar`) è inferiore a 9
- per ogni livello di `quality`, calcolare
  - il `pH` massimo nella colonna `pHMax`
  - la media di `density` nella colonna `densityAvg`
- ordinare quindi per livello di `quality` crescente

Esempio

| quality |  pHMax | densityAvg |
|--------|------|------|
| 1 | 3.5 | 0.99 |
| 2 | 3.6 | 0.98 |
| ... | ... |... |

- salvare il risultato in formato parquet nel seguente path `/exam/221017_wine_part_2.parquet` (⚠ i nomi delle colonne devono coincidere con quelli indicati nella traccia)

### IoT Devices 📶

Il file `databricks-datasets/iot/iot_devices.json` raccoglie dati rilevati da dispositivi IoT in data 20 marzo 2016. La colonna `timestamp` contiene lo [Unix time](https://en.wikipedia.org/wiki/Unix_time) espresso però in **millisecondi**.

- Dalla colonna `timstamp`, estrarre quindi l'ora e il minuto del giorno.
- Calcolare quindi il conteggio dei record per ogni coppia ora e minuto.
- Ordinare per ora e minuti crescenti.

Esempio

| hour |  minute | count |
|--------|------|------|
| 0 | 4 | 99 |
| 1 | 2 | 8 |
| ... | ... |... |

- Salvare il risultato in formato parquet nel seguente path `/exam/221017_iot.parquet` (⚠ i nomi delle colonne devono coincidere con quelli indicati nella traccia)

In [0]:
from pyspark.sql.types import StructType, StructField, DoubleType, IntegerType

schema = StructType(
    [
        StructField("fixed acidity",DoubleType()),
        StructField("volatile acidity",DoubleType()),
        StructField("citric acid",DoubleType()),
        StructField("residual sugar",DoubleType()),
        StructField("chlorides",DoubleType()),
        StructField("free sulfur dioxide",DoubleType()),
        StructField("total sulfur dioxide",DoubleType()),
        StructField("density",DoubleType()),
        StructField("pH",DoubleType()),
        StructField("sulphates",DoubleType()),
        StructField("alcohol",DoubleType()),
        StructField("quality",IntegerType())
    ]
)

In [0]:
df = spark.read.csv(
  "/databricks-datasets/wine-quality/winequality-white.csv",
    sep=';',
    header=True,
    schema=schema
)

In [0]:
from pyspark.sql.functions import col

df.groupBy("quality")\
    .count()\
    .orderBy(df.quality.asc())\
    .write.format("parquet")\
    .save("/exam/221017_wine_part_1.parquet")

In [0]:
df.where(col("residual sugar") < 9)\
    .groupBy("quality")\
    .agg(
        {
            "pH": "max",
            "density": "avg"
        }
    )\
    .withColumnRenamed("avg(density)", "densityAvg")\
    .withColumnRenamed("max(pH)", "pHMax")\
    .orderBy(df.quality.asc())\
    .write.format("parquet")\
    .save("/exam/221017_wine_part_2.parquet")

In [0]:
from pyspark.sql.functions import to_timestamp, year, month, minute

spark.read.format("json").load("/databricks-datasets/iot/iot_devices.json")\
    .withColumn("datetime", to_timestamp(col("timestamp") / 1000))\
    .withColumn("hour", hour(col("datetime")))\
    .withColumn("minute", minute(col("datetime")))\
    .groupBy("hour", "minute")\
    .count()\
    .write.format("parquet")\
    .save("/exam/221017_iot.parquet")